In [121]:
from qdrant_client import QdrantClient, models

In [102]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [103]:
import numpy as np
from fastembed import TextEmbedding

query = 'I just discovered the course. Can I join now?'
model_handle = "jinaai/jina-embeddings-v2-small-en"
embedding_model = TextEmbedding(model_name=model_handle)


# Embed the query (returns a list with one vector)
embeddings = list(embedding_model.embed(query))

#print(embeddings)
len(embeddings[0])
#min(embeddings[0])
array_size_512 = np.zeros(512)
array_size_512 = embeddings[0]
print(np.min(array_size_512))

-0.11726373022974847


In [104]:
import numpy as np
from fastembed import TextEmbedding
doc = 'Can I still join the course after the start date?'
model_handle = "jinaai/jina-embeddings-v2-small-en"
embedding_model = TextEmbedding(model_name=model_handle)
doc_embeddings = list(embedding_model.embed(doc))
print(doc_embeddings[0])
#len(embeddings[0])
#min(embeddings[0])
#array_size_512 = np.zeros(512)
#array_size_512 = embeddings
#print(np.min(array_size_512)) 

[-5.45304133e-02 -7.83451779e-02  3.13610405e-02  2.34234822e-02
 -3.06321542e-02 -5.51041722e-02  2.24726469e-02  1.50386032e-02
 -1.81159762e-02  1.80595373e-02 -4.80890398e-03 -7.57395660e-02
 -4.87271651e-03 -9.07717901e-02  4.83940937e-02 -4.26317272e-02
 -2.61462449e-02 -4.48129083e-02 -2.06926031e-02 -3.79054511e-02
 -3.30329200e-02 -8.02849396e-03  1.18560490e-02 -3.83721080e-02
  7.26378360e-02 -5.82811751e-02 -7.30890342e-02  3.40911512e-02
  5.60705435e-02  5.02684926e-02 -5.99293455e-02  5.33016467e-02
 -1.79524656e-02  4.17742530e-03 -2.45427787e-02  1.32288183e-02
  4.64177968e-02  1.09963575e-02 -1.01499833e-02 -5.77523997e-03
 -2.83978587e-04  4.91496389e-02  1.11214839e-01 -2.76306232e-02
 -6.78923645e-02 -1.85928277e-02 -1.23963212e-01  1.25745928e-02
  1.65298537e-02  7.22241271e-03 -2.77237382e-02  5.18073706e-02
 -5.98604913e-02  5.27339588e-02  1.03844065e-02  4.82091147e-02
 -2.29023363e-02  1.58800078e-02 -4.80511541e-02  1.02489381e-02
  1.37884565e-02  3.87445

In [105]:
# Create collection
collection_name = "zoomcamp-hw2"
client.create_collection(collection_name=collection_name,vectors_config=models.VectorParams(size=512, distance=models.Distance.COSINE))

True

In [106]:
# Create points with 2 text obj (query and doc)
points = []
identifier = 0
docs = [query, doc]
for each in docs:
    #embedding = list(embedding_model.embed(each))[0]
    point = models.PointStruct(id=identifier, vector=models.Document(text=each, model=model_handle),payload={})
    points.append(point)
    identifier = identifier + 1
    
points

[PointStruct(id=0, vector=Document(text='I just discovered the course. Can I join now?', model='jinaai/jina-embeddings-v2-small-en', options=None), payload={}),
 PointStruct(id=1, vector=Document(text='Can I still join the course after the start date?', model='jinaai/jina-embeddings-v2-small-en', options=None), payload={})]

In [107]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [108]:
retrieved_point0 = client.retrieve(
        collection_name=collection_name,
        ids=[0], # Retrieve point with ID 0
        with_payload=False,
        with_vectors=True,
    )
retrieved_point0[0].vector


[-0.07639464,
 -0.07305555,
 0.05865017,
 0.039267052,
 -0.014179709,
 -0.04683428,
 0.026986282,
 0.03490392,
 0.0015342056,
 -0.011040754,
 0.055755626,
 -0.028836023,
 -0.048723962,
 -0.09909239,
 0.06025796,
 -0.018610543,
 -0.008810406,
 -0.030199492,
 -0.014254169,
 -0.038788427,
 -0.027612565,
 -0.0069495845,
 0.032065343,
 -0.0051783794,
 0.08379599,
 -0.08873297,
 -0.07302605,
 0.059284665,
 0.04078055,
 0.07726848,
 -0.046002954,
 0.03984268,
 0.017482154,
 0.008702663,
 -0.0309291,
 0.02204704,
 0.047647998,
 0.017656632,
 -0.036201373,
 -0.035340827,
 -0.0059253783,
 0.029709125,
 0.08058836,
 0.010713263,
 -0.053819254,
 -0.0096288,
 -0.11726373,
 0.03365464,
 0.007057629,
 0.007701601,
 -0.05174826,
 0.054695547,
 -0.07802899,
 0.04218017,
 0.012625102,
 0.053133912,
 -0.026777722,
 0.016882064,
 -0.037506454,
 0.009131969,
 0.021369688,
 0.016015017,
 -0.05056502,
 0.056220617,
 -0.03668027,
 -0.03854168,
 -0.042125713,
 -0.013692141,
 -0.08585819,
 0.02626408,
 -0.06185

In [134]:
retrieved_point1 = client.retrieve(
        collection_name=collection_name,
        ids=[1], # Retrieve point with ID 0
        with_payload=False,
        with_vectors=True,
    )
retrieved_point1
#retrieved_point1[0].vector

[Record(id=1, payload=None, vector=[-0.054530412, -0.07834518, 0.03136104, 0.023423482, -0.030632155, -0.055104174, 0.022472646, 0.015038603, -0.018115977, 0.018059537, -0.004808904, -0.07573956, -0.0048727165, -0.09077179, 0.048394095, -0.042631727, -0.026146244, -0.044812907, -0.020692604, -0.03790545, -0.03303292, -0.008028494, 0.011856049, -0.038372107, 0.07263783, -0.058281176, -0.07308903, 0.034091152, 0.056070544, 0.050268494, -0.059929345, 0.053301647, -0.017952466, 0.0041774255, -0.024542779, 0.013228818, 0.046417795, 0.010996358, -0.010149984, -0.00577524, -0.0002839786, 0.04914964, 0.11121484, -0.027630623, -0.067892365, -0.018592827, -0.123963214, 0.012574593, 0.016529854, 0.0072224126, -0.027723739, 0.05180737, -0.05986049, 0.052733958, 0.010384407, 0.048209116, -0.022902336, 0.015880007, -0.048051152, 0.010248939, 0.013788456, 0.0387445, -0.057989426, 0.056913916, -0.031520545, -0.026749857, -0.044805445, -0.04097075, -0.09204475, -0.0059611546, -0.054933544, -0.005342236

In [110]:
similarity = np.dot(retrieved_point0[0].vector, retrieved_point1[0].vector)
print(similarity)

0.9008529102612799


In [111]:
import json

models = []
for model in TextEmbedding.list_supported_models():
    modelname, dim =  model["model"], model["dim"]
    models.append({"modelname": modelname, "dim": dim})

In [112]:
models

[{'modelname': 'BAAI/bge-base-en', 'dim': 768},
 {'modelname': 'BAAI/bge-base-en-v1.5', 'dim': 768},
 {'modelname': 'BAAI/bge-large-en-v1.5', 'dim': 1024},
 {'modelname': 'BAAI/bge-small-en', 'dim': 384},
 {'modelname': 'BAAI/bge-small-en-v1.5', 'dim': 384},
 {'modelname': 'BAAI/bge-small-zh-v1.5', 'dim': 512},
 {'modelname': 'mixedbread-ai/mxbai-embed-large-v1', 'dim': 1024},
 {'modelname': 'snowflake/snowflake-arctic-embed-xs', 'dim': 384},
 {'modelname': 'snowflake/snowflake-arctic-embed-s', 'dim': 384},
 {'modelname': 'snowflake/snowflake-arctic-embed-m', 'dim': 768},
 {'modelname': 'snowflake/snowflake-arctic-embed-m-long', 'dim': 768},
 {'modelname': 'snowflake/snowflake-arctic-embed-l', 'dim': 1024},
 {'modelname': 'jinaai/jina-clip-v1', 'dim': 768},
 {'modelname': 'Qdrant/clip-ViT-B-32-text', 'dim': 512},
 {'modelname': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384},
 {'modelname': 'jinaai/jina-embeddings-v2-base-en', 'dim': 768},
 {'modelname': 'jinaai/jina-embeddings-v

In [113]:
import json

EMBEDDING_DIMENSIONALITY = 384

for model in TextEmbedding.list_supported_models():
    if model["dim"] < EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

In [114]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [115]:
documents

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [118]:
len(documents)

5

In [122]:
points = []
id = 3

for doc in documents:
    #print(doc)

    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": doc['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [123]:
points

[PointStruct(id=3, vector=Document(text="Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'course': 'data-engineering-zoomcamp'}),
 PointStruct(id=4, vector=Document(text='Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', model='jinaai/jina-embeddings-v2-small

In [124]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [132]:
pts = [3,4,5,6,7]
res = []
for pt in pts:
    current_retrieved_point = client.retrieve(
        collection_name=collection_name,
        ids=[pt], # Retrieve point with ID pt
        with_payload=False,
        with_vectors=True,
    )
    #print(retrieved_point0[0].vector)
    #print(current_retrieved_point[0].vector)
    print(f"Fetched pt: {pt} current_retrieved_point {current_retrieved_point[0].vector}")
    similarityQ3 = np.dot(retrieved_point0[0].vector, current_retrieved_point[0].vector)
    res.append(similarityQ3)
    

Fetched pt: 3 current_retrieved_point [-0.024952471, -0.039645385, -0.004376747, 0.02958303, -0.012030076, -0.0069325007, -7.656725e-05, 0.06151213, -0.028510582, -0.06709609, 0.0065319235, -0.05095759, -0.05012441, -0.058327634, 0.0954579, -0.05410399, -0.032038596, -0.046956033, -0.0616868, -0.09039345, -0.033534914, -0.029053388, 0.044497225, -0.07000456, 0.042759545, -0.07019891, -0.08680373, 0.08142058, 0.07218799, 0.01894729, -0.059557684, 0.010949329, -0.018051147, 0.046081778, -0.029758869, 0.034579217, 0.040522154, 0.017594993, -0.027704153, -0.016721169, -0.05028602, 0.013497159, 0.09174102, 0.008328603, -0.07495525, 0.0023509024, -0.04665368, 0.012956048, 0.02577389, 0.005871563, -0.047506247, 0.0141338315, -0.10499458, 0.031179463, -0.029514957, 0.06340931, -0.012428459, 0.029035192, -0.024678705, 0.0065530515, -0.001689952, 0.04806843, -0.0451916, 0.06588197, 0.0007003945, -0.046943318, -0.0064393315, -0.022500765, -0.11023053, 0.03249054, -0.10615784, 0.030156547, 0.06316

In [124]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [133]:
print(res)

[np.float64(0.7629684545728064), np.float64(0.8182378183427668), np.float64(0.8085397347508796), np.float64(0.7133078685570795), np.float64(0.7304499266693546)]


In [137]:
#Q3
# query = 'I just discovered the course. Can I join now?'
# text='Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.
res[1]

np.float64(0.8182378183427668)

In [138]:
#Q4 Now let's calculate a new field, which is a concatenation of question and text:
# full_text = doc['question'] + ' ' + doc['text']
# Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?


# Q4 create

points = []
id = 8

for doc in documents:
    #print(doc)
    full_text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=full_text, model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "section": doc['section'],
            "course": doc['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [139]:
# Q4 upsert

client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [140]:
# Q4 retrieve

pts = [8,9,10,11,12]
res = []
for pt in pts:
    current_retrieved_point = client.retrieve(
        collection_name=collection_name,
        ids=[pt], # Retrieve point with ID pt
        with_payload=False,
        with_vectors=True,
    )
    #print(retrieved_point0[0].vector)
    #print(current_retrieved_point[0].vector)
    print(f"Fetched pt: {pt} current_retrieved_point {current_retrieved_point[0].vector}")
    similarityQ4 = np.dot(retrieved_point0[0].vector, current_retrieved_point[0].vector)
    res.append(similarityQ4)

Fetched pt: 8 current_retrieved_point [-0.039587475, -0.0690407, 0.01146618, 0.026260396, -0.0247341, -0.024720693, 0.003439709, 0.042785924, -0.024162944, -0.026199237, -0.014921459, -0.07881787, -0.024740098, -0.07740145, 0.07686429, -0.046568375, -0.030845456, -0.044565722, -0.04481413, -0.060655624, -0.036711335, -0.018000936, 0.031952284, -0.06805735, 0.059340086, -0.0710709, -0.08380715, 0.07134353, 0.07235555, 0.024347816, -0.055102453, 0.02945919, -0.018308265, 0.025609974, -0.031728674, 0.027675094, 0.04963585, 0.013883948, -0.020100309, -0.0028251538, -0.029371914, 0.028263265, 0.100648426, 0.0028943631, -0.07165614, 0.00076785556, -0.08024243, 0.011240777, 0.0255504, 0.015388169, -0.03343803, 0.02859502, -0.07989595, 0.034472, -0.007653952, 0.0663229, -0.010571317, 0.024312276, -0.040355776, -0.0028037447, -0.0017486297, 0.030156884, -0.05380364, 0.055337228, -0.010549605, -0.036919657, -0.030680662, -0.027392665, -0.10284295, 0.016560763, -0.09235248, 0.019417338, 0.0747836

In [142]:
print(res)

[np.float64(0.851454330199421), np.float64(0.8436594138456714), np.float64(0.8408287039703563), np.float64(0.7755157971735214), np.float64(0.8086007988045588)]
